In [373]:
import pandas as pd
import networkx as nx
import random
import math
from collections import defaultdict

# minumum bant genişliğini ayarla


In [374]:
nodes = pd.read_csv("nodedata.csv")
edges = pd.read_csv("edgedata.csv")
demands = pd.read_csv("demanddata.csv")
global c 
c = 0


In [375]:
def build_graph_from_csv(node_csv, edge_csv):
    print(f"Veriler yükleniyor: {node_csv} ve {edge_csv}...")
    G = nx.Graph()

    try:
        nodes = pd.read_csv(node_csv, sep=';', decimal=',')
        edges = pd.read_csv(edge_csv, sep=';', decimal=',')
    except FileNotFoundError:
        print("KRİTİK HATA: CSV dosyaları bulunamadı.")
        return None

    for _, n in nodes.iterrows():
        G.add_node(
            int(n["node_id"]),
            s_ms=float(n["s_ms"]),
            r_node=float(n["r_node"])
        )

    for _, e in edges.iterrows():
        G.add_edge(
            int(e["src"]),
            int(e["dst"]),
            capacity_mbps=float(e["capacity_mbps"]),
            delay_ms=float(e["delay_ms"]),
            r_link=float(e["r_link"])
        )

    print(f"Ağ Yüklendi: {G.number_of_nodes()} node, {G.number_of_edges()} edge")
    return G



In [376]:
# ---------- 2. LOAD DEMANDS (ZORUNLU) ----------
def load_demands(demand_csv):
    try:
        return pd.read_csv(demand_csv, sep=';', decimal=',')
    except FileNotFoundError:
        print(f"KRİTİK HATA: '{demand_csv}' dosyası bulunamadı.")
        return None
    except Exception as e:
        print(f"HATA: Talep dosyası okunurken hata: {e}")
        return None



In [377]:
# ---------- 3. YOL KONTROLÜ----------
def is_valid_path(G, path):
    """Yolun fiziksel olarak kopuk olup olmadığını kontrol eder."""
    if not path or len(path) < 2: return False
    for i in range(len(path) - 1):
        if not G.has_edge(path[i], path[i+1]):
            return False
    return True
    

In [378]:
w_d = 2.0
w_r = 5.0          
w_u = 150.0 
c = 0

w_total = w_d + w_r + w_u         # Wdelay + Wreliability + Wresource = 1 

w_d = w_d/w_total
w_r = w_r/w_total
w_u = w_u/w_total

def compute_reward(G, u, v):
    edge = G[u][v]
    node = G.nodes[v]

    delay = edge["delay_ms"] + node["s_ms"]
    resource = 1000 / edge["capacity_mbps"]   # 1gbps yerine 1000mbps 
    reliability =-math.log(edge["r_link"]) - math.log(node["r_node"]) # ReliabilityCost(P)=X(i,j)∈P[−log(LinkReliabilityij)]+Xk∈P[−log(N odeReliabilityk)]

    delay_norm = delay / edges.  #max_delay
    resource_norm = resource / 8.0   #max_resource
    reliability_norm = reliability / 0.06   #max_reliability

    # hepsi minimize 
    
    total_cost = (    # reward = -cost      reward = +benefit
        w_d * delay +
        w_r * resource +
        w_u * reliability
    )
    
    # reward = reward = 1 / (1 + total_cost)
    reward = -total_cost


    print(f"delay: {delay:.2f}, resource: {resource:.2f}, reliability: {reliability:.2f}")
    print(f"delay_cost: {(w_d * delay):.3f}, resource_cost: {(w_r * resource):.3f}, reliability_cost:{(w_u * reliability):.3f}, total_cost: {total_cost:.2f}, reward: {reward:.2f}")
    print(f"w_delay: {w_d:.3f}, w_resource: {w_r:.3f}, w_reliability:{w_u:.3f}")
    global c
    c = c +1
    return reward


SyntaxError: invalid syntax (4277603926.py, line 20)

In [379]:
ALPHA = 0.1        # learning rate
GAMMA = 0.9        # discount
EPSILON = 0.2      # exploration
EPISODES = 20000
MAX_STEPS = 50

Q = defaultdict(lambda: defaultdict(float))



In [380]:
def train_q_learning(G, demands):
    for episode in range(EPISODES):

        d = demands.sample(1).iloc[0]
        current = int(d["src"])
        current = int(d["src"])
        target = int(d["dst"])



        for _ in range(MAX_STEPS):

            neighbors = list(G.neighbors(current))
            if not neighbors:
                break

            # ε-greedy
            if random.random() < EPSILON:
                next_node = random.choice(neighbors)
            else:
                next_node = max(
                    neighbors,
                    key=lambda n: Q[(current, target)][n]
                )
            global c   
            
            r= compute_reward(G, current, next_node)
            
            print(c)

            if next_node == target:
                r += 100

            future_neighbors = list(G.neighbors(next_node))
            best_future = 0
            if future_neighbors:
                best_future = max(
                    Q[(next_node, target)][n]
                    for n in future_neighbors
                )

            Q[(current, target)][next_node] += ALPHA * (
                r + GAMMA * best_future
                - Q[(current, target)][next_node]
            )

            current = next_node

            if current == target:
                break

        if episode % 2000 == 0:
            print(f"Episode {episode} tamamlandı")

    print("Eğitim bitti.")


In [381]:
def extract_policy(Q):
    policy = {}

    for (current, target), actions in Q.items():
        if actions:
            policy[(current, target)] = max(
                actions,
                key=actions.get
            )

    return policy


In [382]:
def get_best_path(policy, src, dst, max_hops=50):
    path = [src]
    current = src

    for _ in range(max_hops):
        if current == dst:
            break

        next_node = policy.get((current, dst))
        if next_node is None:
            break

        path.append(next_node)
        current = next_node

    return path


In [383]:
if __name__ == "__main__":

    G = build_graph_from_csv(
        "nodedata.csv",
        "edgedata.csv"
    )

    demands = load_demands("demanddata.csv")

    train_q_learning(G, demands)

    policy = extract_policy(Q)

    # test
    test_src = int(demands.iloc[0]["src"])
    test_dst = int(demands.iloc[0]["dst"])

    path = get_best_path(policy, test_src, test_dst)
    print("Bulunan yol:", path)


Veriler yükleniyor: nodedata.csv ve edgedata.csv...
Ağ Yüklendi: 250 node, 12452 edge


KeyError: 'delay_ms'

In [ ]:
policy

In [ ]:
policy_edges = []

for s, a in policy.items():
    policy_edges.append((s, a))


In [ ]:
pos = nx.spring_layout(G, seed=42)

nx.draw(G, pos, node_size=20, alpha=0.2)
nx.draw_networkx_edges(
    G, pos,
    edgelist=policy_edges,
    edge_color="red",
    width=2
)

plt.show()
